## Credit Suisse

In [138]:
import pandas as pd
import itertools
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [139]:
missing_values = ["n/a", "na", "--", ' ', "NAN"]
data = pd.read_csv("../resources/Data.csv", na_values = missing_values)

In [140]:
data.head()

,QTR,AC6,AC6.1,AC6.2,AC6.3,AC5,AC5.1,AC6.4,AC6.5,AC6.6,...,AC6.16,AC6.17,AC6.18,AC6.19,AC6.20,AC7.4,AC7.5,AC2.5,AC2.6,Status
0,NaN,V61,V62,V63,V64,V521,V56,V65,V66,V67,...,V621,V619,V614,V615,V613,V75,V76,V24,V21,NaN
1,NaN,NOTCHECKED,NOTCHECKED,CHECKED,NOTCHECKED,NOTCHECKED,NOTCHECKED,NOTCHECKED,NOTCHECKED,NOTCHECKED,...,NOTCHECKED,NOTCHECKED,NOTCHECKED,NOTCHECKED,CHECKED,NOTCHECKED,CHECKED,CHECKED,NOTCHECKED,NaN
2,2007Q1,14.83701892,99.39895748,62.94080546,101.764,1.674048473,3.82953486,1.376969231,1.734046154,1.734892308,...,2.882427354,2.574383623,4.151536606,1.851110209,0.293229125,92.44514042,2852.292845,192.4493716,6466.172493,NOTCHECKED
3,2007Q2,14.90709017,91.33259965,58.44709848,101.1369231,3.004369667,3.82953486,1.400923077,1.547769231,1.698153846,...,2.968603516,2.680464049,4.151519695,1.851584132,0.293911596,91.0359477,2877.140392,202.1334248,6720.868911,NOTCHECKED
4,2007Q3,23.21522878,117.8079253,87.46058542,98.30738462,2.909346725,3.342410308,2.126230769,1.973061538,2.402969231,...,2.977815816,2.701522102,4.228532862,1.869284404,0.291331291,108.3190147,2668.667999,201.2074925,7548.173885,NOTCHECKED


In [141]:
data.shape

(50, 78)

In [142]:
col_names = []
for i in data.columns:
    col_names.append(i)
variables = sorted(col_names)[:76]
print(variables)

['AC1', 'AC1.1', 'AC1.2', 'AC1.3', 'AC1.4', 'AC2', 'AC2.1', 'AC2.2', 'AC2.3', 'AC2.4', 'AC2.5', 'AC2.6', 'AC3', 'AC3.1', 'AC3.2', 'AC3.3', 'AC3.4', 'AC4', 'AC4.1', 'AC4.2', 'AC4.3', 'AC4.4', 'AC4.5', 'AC5', 'AC5.1', 'AC5.10', 'AC5.11', 'AC5.12', 'AC5.13', 'AC5.14', 'AC5.15', 'AC5.16', 'AC5.17', 'AC5.18', 'AC5.2', 'AC5.3', 'AC5.4', 'AC5.5', 'AC5.6', 'AC5.7', 'AC5.8', 'AC5.9', 'AC6', 'AC6.1', 'AC6.10', 'AC6.11', 'AC6.12', 'AC6.13', 'AC6.14', 'AC6.15', 'AC6.16', 'AC6.17', 'AC6.18', 'AC6.19', 'AC6.2', 'AC6.20', 'AC6.3', 'AC6.4', 'AC6.5', 'AC6.6', 'AC6.7', 'AC6.8', 'AC6.9', 'AC7', 'AC7.1', 'AC7.2', 'AC7.3', 'AC7.4', 'AC7.5', 'AC8', 'AC8.1', 'AC8.2', 'AC8.3', 'AC8.4', 'AC8.5', 'AC8.6']


In [143]:
# Dictionary storing the data type (Checked or Not-checked)
data_type = {}
for i in variables:
    data_type[i] = data[i][1]

In [144]:
# Filters the data for the missing values (for e.g. ' ', NAN, na, n/a)
def Isnan(x):
    x = np.array(data[x][2:])
    x_cleaned = [float(i) for i in x if str(i) != 'nan']
    if len(x) == len(x_cleaned):
        return("No missing values found")
    else:
        return("missing values found")

In [145]:
# Filters the data for the missing values (for e.g. ' ', NAN, na, n/a)
def Cleaner(x):
    x = np.array(data[x][2:])
    x_cleaned = [float(i) for i in x if str(i) != 'nan']
    return x_cleaned

In [146]:
# checked & not_checked are list storing checked & not_checked variables respestively
checked = []
not_checked = []
for i in variables:
    if data_type[i] == 'CHECKED':
        checked.append(i)
    else:
        not_checked.append(i)

In [147]:
not_checked_no_missingvalue = []
for i in not_checked:
    if Isnan(i) == 'No missing values found':
        not_checked_no_missingvalue.append(i)

In [148]:
len(checked),len(not_checked)

(9, 67)

In [149]:
result = pd.DataFrame(columns = variables)
for i in variables:
    result[i] = list(np.arange(1,49))
result.insert(0, "QTR", list(data['QTR'][2:]),True)     

## Outlier detection with Local Outlier Factor (LOF)

The Local Outlier Factor (LOF) algorithm is an unsupervised anomaly detection method which computes the local density deviation of a given data point with respect to its neighbors. It considers as outliers the samples that have a substantially lower density than their neighbors. This example shows how to use LOF for outlier detection which is the default use case of this estimator in scikit-learn. Note that when LOF is used for outlier detection it has no predict, decision_function and score_samples methods. See User Guide <outlier_detection>: for details on the difference between outlier detection and novelty detection and how to use LOF for novelty detection.

The number of neighbors considered (parameter n_neighbors) is typically set 1) greater than the minimum number of samples a cluster has to contain, so that other samples can be local outliers relative to this cluster, and 2) smaller than the maximum number of close by samples that can potentially be local outliers. In practice, such informations are generally not available, and taking n_neighbors=20 appears to work well in general.

In [150]:
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

In [151]:
def LOF_values(X):
    X = Cleaner(str(X))
    X = list(zip(X,X))
    X = np.array(X)
    clf = LocalOutlierFactor(n_neighbors=20, algorithm = 'auto')
    y_pred = clf.fit_predict(X)
    return(y_pred)

In [152]:
# Storing the results in results:
for i in not_checked_no_missingvalue:
    result[i] = list(LOF_values(i))

In [153]:
result.head()

,QTR,AC1,AC1.1,AC1.2,AC1.3,AC1.4,AC2,AC2.1,AC2.2,AC2.3,...,AC7.3,AC7.4,AC7.5,AC8,AC8.1,AC8.2,AC8.3,AC8.4,AC8.5,AC8.6
0,2007Q1,1,1,1,1,1,-1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2007Q2,1,1,1,1,2,-1,1,1,2,...,1,2,2,2,2,1,1,1,1,1
2,2007Q3,1,1,1,1,3,-1,1,1,3,...,1,3,3,3,3,1,1,1,1,1
3,2007Q4,1,1,1,1,4,-1,1,1,4,...,1,4,4,4,4,1,1,-1,1,1
4,2008Q1,1,1,1,1,5,1,1,1,5,...,1,5,5,5,5,1,-1,-1,-1,1


In [154]:
col_with_missingval = []
for i in variables:
    if Isnan(i) == "missing values found":
        col_with_missingval.append(i)

## Estimating Missing Values

In [155]:
missing_value_df = pd.DataFrame(data[i][2:] for i in col_with_missingval)
missing_value_df = missing_value_df.transpose()
missing_value_df = missing_value_df.astype(float)
missing_value_df = missing_value_df.interpolate()

In [156]:
def LOF_missing_value(X):
    X = list(zip(X,X))
    X = np.array(X)
    clf = LocalOutlierFactor(n_neighbors=20, algorithm = 'auto')
    y_pred = clf.fit_predict(X)
    return(y_pred)

In [157]:
for i in col_with_missingval:
    result[i] = list(LOF_missing_value(missing_value_df[i]))

In [158]:
for i in checked:
    result[i] = list(np.linspace(1, 1, num=48))

## Isolation Forest

##### An example using :class:sklearn.ensemble.IsolationForest for anomaly detection.

##### The IsolationForest 'isolates' observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature.

##### Since recursive partitioning can be represented by a tree structure, the number of splittings required to isolate a sample is equivalent to the path length from the root node to the terminating node.

##### This path length, averaged over a forest of such random trees, is a measure of normality and our decision function.

##### Random partitioning produces noticeable shorter paths for anomalies. Hence, when a forest of random trees collectively produce shorter path lengths for particular samples, they are highly likely to be anomalies.

In [159]:
from sklearn.ensemble import IsolationForest

In [210]:
def Isolation_forest(X):
    X = Cleaner(str(X))
    X = list(zip(X,X))
    X = np.array(X)
    rng = np.random.RandomState(42)
    clf = IsolationForest(behaviour='new', max_samples=100, random_state=rng, contamination='auto')
    clf.fit(X)
    y_pred = clf.fit_predict(X)
    return(y_pred)

In [211]:
def Isolation_forest_missingval(X):
    X = list(zip(X,X))
    X = np.array(X)
    rng = np.random.RandomState(42)
    clf = IsolationForest(behaviour='new', max_samples=100, random_state=rng, contamination='auto')
    clf.fit(X)
    y_pred = clf.fit_predict(X)
    return(y_pred)

In [212]:
# creating as data frame to store the results
result_if = pd.DataFrame(columns = variables)
for i in variables:
    result_if[i] = list(np.arange(1,49))
result_if.insert(0, "QTR", list(data['QTR'][2:]),True)

# Storing the obtained results in the dataframe:
for i in not_checked_no_missingvalue:
    result_if[i] = list(Isolation_forest(i))

# storing the known results:
for i in checked:
    result_if[i] = list(np.linspace(1, 1, num=48))
    
# Storing the results for the missing values
for i in col_with_missingval:
    result_if[i] = list(Isolation_forest_missingval(missing_value_df[i]))

## Outliar based on Quantile



This is the simplest, nonparametric outlier detection method in a one dimensional feature space. Here outliers are calculated by means of the IQR (InterQuartile Range).

The first and the third quartile ($Q1$, $Q3$) are calculated. An outlier is then a data point $x_i$ that lies outside the interquartile range. That is:

$x_i > Q3 + k*IQR$ or $x_i < Q1 - k*IQR$

where, $IQR = Q3-Q1$

Using the interquartile multiplier value k=1.5, the range limits are the typical upper and lower whiskers of a box plot.


In [163]:
def IQR(X, k):
    X = Cleaner(str(X))
    q1 = np.quantile(X, 0.25)
    q3 = np.quantile(X,0.75)
    iqr = q3-q1
    y_pred = []
    for i in X:
        if i>q3+k*iqr or i<q1-k*iqr:
            y_pred.append(-1)
        else:
            y_pred.append(1)
    return y_pred

In [164]:
def IQR_missingval(X, k):
    q1 = np.quantile(X, 0.25)
    q3 = np.quantile(X,0.75)
    iqr = q3-q1
    y_pred = []
    for i in X:
        if i>q3+k*iqr or i<q1-k*iqr:
            y_pred.append(-1)
        else:
            y_pred.append(1)
    return y_pred

In [165]:
# creating as data frame to store the results
result_iqr = pd.DataFrame(columns = variables)
for i in variables:
    result_if[i] = list(np.arange(1,49))
result_iqr.insert(0, "QTR", list(data['QTR'][2:]),True)     

# Storing the obtained results in the dataframe:
for i in not_checked_no_missingvalue:
    result_iqr[i] = list(IQR(i,1.5))
# storing the known results:
for i in checked:
    result_iqr[i] = list(np.linspace(1, 1, num=48))
    
# Storing the results for the missing values
for i in col_with_missingval:
    result_iqr[i] = list(IQR_missingval(missing_value_df[i],1.5))    

In [166]:
data_estim = data
for i in col_with_missingval:
    data_estim[i] = missing_value_df[i]

### Computing Emperical CDF

In [167]:
def cumf(x):
    sorted_x = sorted(x)
    ecdf = []
    for i in x:
        count = 0
        for j in sorted_x:
            if j <= i:
                count += 1
        ecdf.append(count/len(x))
    return ecdf    

In [168]:
ecdf_df = pd.DataFrame(columns = variables)
for i in variables:
    ecdf_df[i] = cumf(data_estim[str(i)][2:])

In [169]:
for i in variables:
    for j in range(len(data_estim[i][2:])):
        ecdf_df[str(i)][j] = min(ecdf_df[str(i)][j],1-ecdf_df[str(i)][j])

In [170]:
for i in variables:
    for j in range(len(data_estim[2:])):
        ecdf_df[str(i)][j] = ecdf_df[str(i)][j]/0.5

In [171]:
ecdf_df.insert(0, "QTR", list(data['QTR'][2:]),True)

In [172]:
checked_row = []
for i,x in data.iterrows():
    if data.loc[i,"Status"] == "CHECKED":
        checked_row.append(i)

In [183]:
def skip_var(l):
    #ls=collections.deque(ls)
    df3=pd.DataFrame()
    l=np.array(l)
    #l=l[np.isnan(l)]
    ls=list(l.astype(np.float))
    skip_vars=[]
    variables=[]
    for i in np.arange(0,len(ls),1):
            #deque_slice = collections.deque(itertools.islice(ls, 0, (len(list(ls))-1)))
            if i==0:
                skip_vars.append(np.var(ls[1:]))
                variables.append(ls[0])
            elif i==len(ls):
                skip_vars.append(np.var(ls[:len(ls)]))
            else:
                skip_vars.append(np.var(ls[:i]+ls[i+1:]))
                variables.append(ls[i])
                #ls=ls.rotate(1)
    df3['variables']=variables
    df3['skip_vars']=skip_vars
    #df3['diff']=np.var(l)-list(df3['skip_vars'])
    return df3
        
    

# Getting the diff values for all the columns in data

df_diff=pd.DataFrame()
for i in data.columns:
    if( i!='QTR' and i!='Status'):
        df_diff_temp=skip_var(list(data[i][2:50]))
        df_diff_temp['diff']=np.var(np.array(data[i][2:50]).astype(np.float))-list(df_diff_temp['skip_vars'])
        df_diff[i]=df_diff_temp['diff']
    
   

# Getting the outlier decision for the entire matrix

def outlier(df):
    df_sign=pd.DataFrame(0, index=np.arange(len(df)), columns=df.columns)
    for i in df.columns:
        if(i!='QTR' and i!='Status'):
            m=np.mean(list(df[i]))
            std=np.std(list(df[i]))
            indices1=df[df[i]>= m+(3*std)].index.values
            indices2=df[df[i]<= m-(3*std)].index.values
            indices1=list(set(indices1)-set(np.arange(26,37,1)))##keeping out checked rows
            indices2=list(set(indices1)-set(np.arange(26,37,1)))
            if(data[i][1]=='NOTCHECKED'):##keeping out checked columns
                df_sign[i][indices1]=1
                df_sign[i][indices2]=1
                #df_sign[i][list(set(df_sign[i].index.values)-set(indices1+indices2+[0,1]))]=0  
            #else:
                #df_sign[i][list(set(df_sign[i].index.values)-set([0,1]))]=0
    return df_sign
        
        

df_outlier=outlier(df_diff)

df_final=data
for i in df_final.columns:
    if(i!='QTR' and i!='Status'):
        df_final[i][2:50]=df_outlier[i]
        

#### Getting the diff values for all the columns in data

In [184]:
df_diff=pd.DataFrame()
for i in data.columns:
    if( i!='QTR' and i!='Status'):
        df_diff_temp=skip_var(list(data[i][2:50]))
        df_diff_temp['diff']=np.var(np.array(data[i][2:50]).astype(np.float))-list(df_diff_temp['skip_vars'])
        df_diff[i]=df_diff_temp['diff']
    
   

#### Getting the outlier decision for the entire matrix

In [200]:
def outlier(df):
    df_sign=pd.DataFrame(0, index=np.arange(len(df)), columns=df.columns)
    for i in df.columns:
        if(i!='QTR' and i!='Status'):
            m=np.mean(list(df[i]))
            std=np.std(list(df[i]))
            indices1=df[df[i]>= m+(3*std)].index.values
            indices2=df[df[i]<= m-(3*std)].index.values
            indices1=list(set(indices1)-set(np.arange(26,37,1)))##keeping out checked rows
            indices2=list(set(indices1)-set(np.arange(26,37,1)))
            if(data[i][1]=='NOTCHECKED'):##keeping out checked columns
                df_sign[i][indices1]=1
                df_sign[i][indices2]=1
                #df_sign[i][list(set(df_sign[i].index.values)-set(indices1+indices2+[0,1]))]=0  
            #else:
                #df_sign[i][list(set(df_sign[i].index.values)-set([0,1]))]=0
    return df_sign
        
        

df_outlier=outlier(df_diff)

df_final=data
for i in df_final.columns:
    if(i!='QTR' and i!='Status'):
        df_final[i][2:50]=df_outlier[i]

In [201]:
df_final.shape

df_final = df_final[2:].reset_index()

df_final = df_final.drop(["index","QTR","Status"],axis = 1)

df_final = df_final.reindex(sorted(df_final.columns), axis=1)

df_final.shape

df_final.insert(0, "QTR", list(data['QTR'][2:]),True)   

final_df = pd.DataFrame(np.rec.fromarrays((result.values, result_if.values, result_iqr.values,ecdf_df.values,df_final.values)).tolist(),columns=result.columns,index=result.index)

(50, 78)

## final data frame

In [214]:
final_df

,QTR,AC1,AC1.1,AC1.2,AC1.3,AC1.4,AC2,AC2.1,AC2.2,AC2.3,...,AC7.3,AC7.4,AC7.5,AC8,AC8.1,AC8.2,AC8.3,AC8.4,AC8.5,AC8.6
0,"(2007Q1, 2007Q1, 2007Q1, 2007Q1, 2007Q1)","(1, 1, 1, 0.041666666666666664, 0)","(1, 1, 1, 0.08333333333333326, 0)","(1, 1, 1, 0.41666666666666674, 0)","(1, 1, 1, 0.25, 0)","(-1, -1, -1, 0.04166666666666674, 0.0)","(-1, 1, 1, 0.125, 0)","(1, 1, 1, 0.5416666666666666, 0)","(1, 1, 1, 0.9583333333333334, 0)","(1, 1, 1, 0.5833333333333333, 0.0)",...,"(1, 1, 1, 0.9166666666666666, 1)","(1, 1, 1, 0.375, 0.0)","(1.0, 1.0, 1.0, 0.875, 0)","(1.0, 1.0, 1.0, 0.16666666666666666, 0)","(1, 1, 1, 0.6666666666666666, 0.0)","(1, 1, 1, 0.7916666666666666, 0)","(1, 1, 1, 0.5833333333333334, 0)","(1, 1, 1, 0.041666666666666664, 0)","(1, 1, 1, 0.4166666666666667, 0)","(1, 1, 1, 1.0, 0)"
1,"(2007Q2, 2007Q2, 2007Q2, 2007Q2, 2007Q2)","(1, 1, 1, 0.08333333333333333, 0)","(1, 1, 1, 0.9166666666666666, 0)","(1, 1, 1, 0.6666666666666667, 0)","(1, 1, 1, 0.16666666666666666, 0)","(-1, -1, -1, 0.0, 0.0)","(-1, -1, 1, 0.0, 1)","(1, 1, 1, 0.875, 0)","(1, 1, 1, 0.6666666666666667, 0)","(1, 1, 1, 0.5416666666666667, 0.0)",...,"(1, 1, 1, 0.8333333333333333, 1)","(1, 1, 1, 0.2916666666666667, 0.0)","(1.0, 1.0, 1.0, 0.8333333333333333, 0)","(1.0, 1.0, 1.0, 0.4583333333333333, 0)","(1, 1, 1, 0.75, 0.0)","(1, 1, 1, 0.875, 0)","(1, 1, 1, 0.625, 0)","(1, 1, 1, 0.16666666666666674, 0)","(1, 1, 1, 0.2916666666666667, 0)","(1, 1, 1, 0.9583333333333334, 0)"
2,"(2007Q3, 2007Q3, 2007Q3, 2007Q3, 2007Q3)","(1, 1, 1, 0.125, 0)","(1, 1, 1, 1.0, 0)","(1, 1, 1, 0.875, 0)","(1, 1, 1, 0.3333333333333333, 0)","(-1, -1, 1, 0.08333333333333326, 0.0)","(-1, 1, 1, 0.08333333333333326, 0)","(1, 1, 1, 0.5, 0)","(1, 1, 1, 0.9166666666666667, 0)","(1, 1, 1, 0.625, 0.0)",...,"(1, 1, 1, 0.5833333333333333, 1)","(1, 1, 1, 0.5833333333333333, 0.0)","(1.0, 1.0, 1.0, 0.875, 0)","(1.0, 1.0, 1.0, 0.5416666666666667, 0)","(1, 1, 1, 0.41666666666666674, 0.0)","(1, 1, 1, 0.7083333333333333, 0)","(1, 1, 1, 0.5, 0)","(1, 1, 1, 0.5833333333333334, 0)","(1, 1, 1, 0.75, 0)","(1, 1, 1, 0.9583333333333333, 0)"
3,"(2007Q4, 2007Q4, 2007Q4, 2007Q4, 2007Q4)","(1, 1, 1, 0.16666666666666666, 0)","(1, 1, 1, 0.4583333333333333, 0)","(1, 1, 1, 0.7083333333333333, 0)","(1, 1, 1, 0.375, 0)","(-1, -1, 1, 0.125, 0.0)","(-1, 1, 1, 0.04166666666666674, 0)","(1, 1, 1, 0.6666666666666667, 0)","(1, 1, 1, 0.7916666666666667, 0)","(1, 1, 1, 0.8333333333333333, 0.0)",...,"(1, 1, 1, 0.45833333333333326, 1)","(1, 1, 1, 0.25, 0.0)","(1.0, 1.0, 1.0, 0.9166666666666666, 0)","(1.0, 1.0, 1.0, 0.5, 0)","(1, 1, 1, 0.375, 0.0)","(1, 1, 1, 0.9166666666666667, 0)","(1, 1, 1, 0.5416666666666667, 0)","(-1, 1, 1, 0.875, 0)","(1, 1, 1, 0.6666666666666667, 0)","(1, 1, 1, 0.9166666666666666, 0)"
4,"(2008Q1, 2008Q1, 2008Q1, 2008Q1, 2008Q1)","(1, 1, 1, 0.8333333333333334, 0)","(1, 1, 1, 0.9166666666666667, 0)","(1, 1, 1, 0.16666666666666674, 0)","(1, 1, 1, 0.75, 0)","(-1, -1, 1, 0.16666666666666674, 0.0)","(1, 1, 1, 0.16666666666666674, 0)","(1, 1, 1, 0.8333333333333334, 0)","(1, 1, 1, 0.7083333333333334, 0)","(1, 1, 1, 1.0, 0.0)",...,"(1, 1, 1, 0.9166666666666667, 1)","(-1, -1, 1, 0.125, 0.0)","(1.0, 1.0, 1.0, 0.7083333333333334, 0)","(1.0, 1.0, 1.0, 0.29166666666666674, 0)","(-1, -1, 1, 0.16666666666666674, 0.0)","(1, 1, 1, 0.29166666666666674, 0)","(-1, 1, 1, 0.125, 0)","(-1, 1, -1, 0.9583333333333334, 0)","(-1, 1, 1, 0.08333333333333326, 0)","(1, 1, 1, 0.9166666666666667, 0)"
5,"(2008Q2, 2008Q2, 2008Q2, 2008Q2, 2008Q2)","(1, 1, 1, 0.9583333333333334, 0)","(1, 1, 1, 0.7083333333333333, 0)","(-1, -1, 1, 0.08333333333333333, 1)","(1, 1, 1, 0.5416666666666666, 0)","(1, -1, 1, 0.25, 0.0)","(1, 1, 1, 0.20833333333333326, 0)","(1, 1, 1, 0.7916666666666667, 0)","(1, 1, 1, 0.8333333333333334, 0)","(1, 1, 1, 0.875, 0.0)",...,"(1, 1, 1, 0.7916666666666666, 1)","(-1, -1, 1, 0.16666666666666674, 0.0)","(1.0, 1.0, 1.0, 0.75, 0)","(1.0, 1.0, 1.0, 0.625, 0)","(1, 1, 1, 0.625, 0.0)","(1, 1, 1, 0.8333333333333333, 0)","(1, 1, 1, 0.375, 0)","(1,

In [215]:
elements = []
for i in checked_row:
    for j in checked:
        elements.append(final_df.loc[i,str(j)])

In [216]:
elements = [list(i) for i in elements]

In [217]:
for i in range(len(elements)):
    if elements[i][-1] >= 0.5:
        elements[i][-1] = 1
    else:
        elements[i][-1] = -1

### Giving the weightage to different methods

In [218]:
method1,method2,method3,method4,method5 = 0,0,0,0,0
for i in elements:
    if i[0] == 1:
        method1 += 1
    if i[1] == 1:
        method2 += 1
    if i[2] == 1:
        method3 += 1
    if i[3] == 1:
        method4 += 1
    if i[4] == 1:
        method5 += 1

In [219]:
total = method1 + method2 + method3 + method4 + method5
method1,method2,method3,method4,method5 = method1/total,method2/total,method3/total,method4/total,method5/total

## multypling each tuple of the final dataframe with corresponding weights of the each methods.

In [220]:
for i in variables:
    for j in range(len(final_df[str(i)])):
        final_df[str(i)][j] = (final_df[str(i)][j][0] * method1) + (final_df[str(i)][j][1] * method2) + (final_df[str(i)][j][2] * method3) + (final_df[str(i)][j][3] * method4) + (final_df[str(i)][j][4] * method5)

In [221]:
for i in variables:
    for j in range(len(final_df[str(i)])):
        if final_df[str(i)][j] >= 0:
            final_df[str(i)][j] = 1
        else:
            final_df[str(i)][j] = -1 

## 1 means not outlier and -1 means outlier

In [222]:
for i in range(len(final_df["QTR"])):
    final_df["QTR"][i] = final_df["QTR"][i][0]

In [223]:
final_df

,QTR,AC1,AC1.1,AC1.2,AC1.3,AC1.4,AC2,AC2.1,AC2.2,AC2.3,...,AC7.3,AC7.4,AC7.5,AC8,AC8.1,AC8.2,AC8.3,AC8.4,AC8.5,AC8.6
0,2007Q1,1,1,1,1,-1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2007Q2,1,1,1,1,-1,-1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2007Q3,1,1,1,1,-1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,2007Q4,1,1,1,1,-1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,2008Q1,1,1,1,1,-1,1,1,1,1,...,1,-1,1,1,-1,1,1,-1,1,1
5,2008Q2,1,1,-1,1,1,1,1,1,1,...,1,-1,1,1,1,1,1,1,1,1
6,2008Q3,1,1,1,1,1,1,1,1,1,...,1,-1,1,1,1,1,1,1,1,1
7,2008Q4,1,-1,1,1,1,1,-1,1,1,...,1,-1,1,1,-1,-1,-1,-1,-1,1
8,2009Q1,1,-1,1,1,1,1,-1,-1,1,...,1,-1,1,1,-1,1,-1,-1,-1,1
9,2009Q2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,-1,1,1,1,1,1
